# Sea-Bird Scientific Community Toolkit - CTD Processing (1.0.0)
## Introduction
This python notebook has been written to give Sea-Bird Scientific (SBS) customers and the scientific community at large a set of python code to be used to process and analyze data from the CTD family of SBS instruments. This toolkit was made in conjunction with the Fathom project. Great care has been taken to make sure that the python code that makes up this toolkit and its related packages gives the same results as the Fathom application.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

## Init

The initialization code section below is used to import required libraries and to make a helper function that is used to make scatter charts with the Plotly package.

In [ ]:
# Native imports
import os.path

# Third-party imports
import gsw
import numpy as np
import pandas as pd

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.processing as proc
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz

import example_data.example_coefficients as ec


def plot(
    data, title, x_names, x_bounds={0: [17, 24], 1: [-1, 6], 2: [-1, 6], 3: [-1, 6]}
):
    config = viz.ChartConfig(
        title=title,
        x_names=x_names,
        y_names=["press"],
        z_names=[],
        chart_type="overlay",
        bounds={
            "x": x_bounds,
            # 'y': { 0: [29, 32], 1: [29, 32], 2: [29, 32], 3: [29, 32] }
        },
        plot_loop_edit_flags=False,
        lift_pen_over_bad_data=True,
    )

    chart_data = viz.ChartData(data, config)
    fig = viz.plot_xy_chart(chart_data, config)
    fig["layout"]["yaxis"]["autorange"] = "reversed"
    fig.update_layout(height=800)
    fig.show()

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into engineering units for the instruments that follow:  
- SBE37SM
- SBE37SMP
- SBE37SMPODO
- SBE37IM
- SBE37IMP
- SBE37IMPODO

Supported output units include temperature, conductivity, pressure, density, potential density, and depth.

In [11]:
hex_file = os.path.join("..","tests", "resources", "test-data", "19plus_V2_CTD-processing_example.hex")

print(hex_file)

raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.SBE19Plus,
    enabled_sensors=[
        id.Sensors.Temperature,
        id.Sensors.Conductivity,
        id.Sensors.Pressure,
        id.Sensors.ExtVolt0,
        id.Sensors.ExtVolt1,
        id.Sensors.ExtVolt2,
        id.Sensors.ExtVolt3,
    ],
)

temperature = conv.convert_temperature(
    temperature_counts_in=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn6130,
    standard='IPTS68',
    units='C',
    use_mv_r=True,
)

pressure = conv.convert_pressure(
    pressure_count=raw_data["pressure"].values,
    compensation_voltage=raw_data["temperature compensation"],
    coefs=ec.pressure_coefs_sn6130,
    units='psia',
)

conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature,
    pressure=pressure,
    coefs=ec.conductivity_coefs_sn6130
)

flag = np.zeros(len(temperature))

tcp_data = np.array([temperature, pressure, conductivity, flag]).transpose()
data = pd.DataFrame(data=tcp_data, columns=["temp", "press", "cond", "flag"])

plot(data=data, title="Data Conversion", x_names=["temp", "cond"], x_bounds={})

..\tests\resources\test-data\19plus_V2_CTD-processing_example.hex
